# Market Basket Analysis

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

---

# Importing Libraries

In [ ]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import networkx as nx
import matplotlib.pyplot as plt
from pandas.plotting import parallel_coordinates
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D
from sklearn.cluster import KMeans
import plotly.graph_objects as go

---

# Data Exploration and Preparation

In [ ]:
purchase_df = pd.read_csv('Assignment-1_Data.csv', sep=';', dtype={'BillNo': str})
purchase_df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,01.12.2010 08:26,"2,55",17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,01.12.2010 08:26,"2,75",17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,01.12.2010 08:26,"3,39",17850.0,United Kingdom


In [ ]:
purchase_df.info()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522064 entries, 0 to 522063
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   BillNo      522064 non-null  object 
 1   Itemname    520609 non-null  object 
 2   Quantity    522064 non-null  int64  
 3   Date        522064 non-null  object 
 4   Price       522064 non-null  object 
 5   CustomerID  388023 non-null  float64
 6   Country     522064 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 27.9+ MB


In [ ]:
purchase_df.isnull().sum()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0
BillNo,0
Itemname,1455
Quantity,0
Date,0
Price,0
CustomerID,134041
Country,0


In [ ]:
purchase_df.nunique()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0
BillNo,21663
Itemname,4185
Quantity,690
Date,19641
Price,1285
CustomerID,4297
Country,30


In [ ]:
# Convert the column to datetime with the specified format
purchase_df['Date'] = pd.to_datetime(purchase_df['Date'], format='%d.%m.%Y %H:%M')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
purchase_df['Price'] = purchase_df['Price'].str.replace(',', '.')
purchase_df['Price'] = purchase_df['Price'].astype(float)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
purchase_df = purchase_df.dropna()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
purchase_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 388023 entries, 0 to 522063
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   BillNo      388023 non-null  object        
 1   Itemname    388023 non-null  object        
 2   Quantity    388023 non-null  int64         
 3   Date        388023 non-null  datetime64[ns]
 4   Price       388023 non-null  float64       
 5   CustomerID  388023 non-null  float64       
 6   Country     388023 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 23.7+ MB


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
purchase_df.isnull().sum()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0
BillNo,0
Itemname,0
Quantity,0
Date,0
Price,0
CustomerID,0
Country,0


In [ ]:
purchase_df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
purchase_df['CustomerID'] = purchase_df['CustomerID'].astype('str')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
purchase_df.info()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<class 'pandas.core.frame.DataFrame'>
Index: 388023 entries, 0 to 522063
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   BillNo      388023 non-null  object        
 1   Itemname    388023 non-null  object        
 2   Quantity    388023 non-null  int64         
 3   Date        388023 non-null  datetime64[ns]
 4   Price       388023 non-null  float64       
 5   CustomerID  388023 non-null  object        
 6   Country     388023 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 23.7+ MB


In [ ]:
purchase_df[['Quantity', 'Price']].describe().round(2)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Quantity,Price
count,388023.00,388023.00
mean,12.89,3.08
std,182.61,21.98
min,1.00,0.00
25%,2.00,1.25
50%,5.00,1.95
75%,12.00,3.75
max,80995.00,8142.75


In [ ]:
purchase_df=purchase_df.loc[purchase_df['Quantity']>0]
purchase_df=purchase_df.loc[purchase_df['Price']>0]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
purchase_df.info()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


<class 'pandas.core.frame.DataFrame'>
Index: 387985 entries, 0 to 522063
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype         
---  ------      --------------   -----         
 0   BillNo      387985 non-null  object        
 1   Itemname    387985 non-null  object        
 2   Quantity    387985 non-null  int64         
 3   Date        387985 non-null  datetime64[ns]
 4   Price       387985 non-null  float64       
 5   CustomerID  387985 non-null  object        
 6   Country     387985 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(4)
memory usage: 23.7+ MB


In [ ]:
purchase_df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [ ]:
transaction = purchase_df.groupby(['BillNo', 'Date'])['Itemname'].apply(lambda x: ', '.join(x)).reset_index()
transaction.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,BillNo,Date,Itemname
0,536365,2010-12-01 08:26:00,"WHITE HANGING HEART T-LIGHT HOLDER, WHITE META..."
1,536366,2010-12-01 08:28:00,"HAND WARMER UNION JACK, HAND WARMER RED POLKA DOT"
2,536367,2010-12-01 08:34:00,"ASSORTED COLOUR BIRD ORNAMENT, POPPY'S PLAYHOU..."
3,536368,2010-12-01 08:34:00,"JAM MAKING SET WITH JARS, RED COAT RACK PARIS ..."
4,536369,2010-12-01 08:35:00,BATH BUILDING BLOCK WORD


In [ ]:
transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18188 entries, 0 to 18187
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   BillNo    18188 non-null  object        
 1   Date      18188 non-null  datetime64[ns]
 2   Itemname  18188 non-null  object        
dtypes: datetime64[ns](1), object(2)
memory usage: 426.4+ KB


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Drop unnecessary columns
transaction.drop(columns=['BillNo', 'Date'], inplace=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Split the 'Itemname' column into separate rows
transaction = transaction.assign(Itemname=transaction['Itemname'].str.split(', ')).explode('Itemname')

# Create a crosstab (binary matrix) of items per transaction
basket_encoded = pd.crosstab(index=transaction.index, columns=transaction['Itemname'])

# Group by the transaction index and convert the counts of items done by pd.crosstab() into a simple presence or absence
basket_encoded = basket_encoded.groupby(level=0).apply(lambda x: x > 0)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


---

# Association Rule Mining

In [ ]:
pip install --upgrade mlxtend

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Association Rule Mining
frequent_itemsets = apriori(basket_encoded, min_support=0.01, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.6, num_itemsets=frequent_itemsets)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# Display information of the rules
print("Association Rules:")
rules.head()

Association Rules:


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(ALARM CLOCK BAKELIKE CHOCOLATE),(ALARM CLOCK BAKELIKE GREEN),0.017154,0.042940,0.011381,0.663462,15.450753,1.0,0.010645,2.843834,0.951602,0.233634,0.648362,0.464253
1,(ALARM CLOCK BAKELIKE CHOCOLATE),(ALARM CLOCK BAKELIKE RED),0.017154,0.047009,0.011986,0.698718,14.863488,1.0,0.011180,3.163119,0.949000,0.229715,0.683856,0.476844
2,(ALARM CLOCK BAKELIKE ORANGE),(ALARM CLOCK BAKELIKE GREEN),0.019298,0.042940,0.011931,0.618234,14.397481,1.0,0.011102,2.506925,0.948855,0.237158,0.601105,0.448041
3,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.047009,0.042940,0.028810,0.612865,14.272468,1.0,0.026792,2.472163,0.975807,0.471223,0.595496,0.641900
4,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.042940,0.047009,0.028810,0.670935,14.272468,1.0,0.026792,2.896054,0.971658,0.471223,0.654703,0.641900


---

# Market Basket Analysis - Support vs. Confidence

In [ ]:
# Convert frozensets to lists for serialization
rules['antecedents'] = rules['antecedents'].apply(list)
rules['consequents'] = rules['consequents'].apply(list)

# Create a scatter plot using plotly.graph_objects
fig = go.Figure()

# Add scatter trace
fig.add_trace(go.Scatter(
    x=rules['support'],
    y=rules['confidence'],
    mode='markers',
    marker=dict(
        size=rules['lift'] * 1,  # Scale size by lift for better visibility
        color=rules['lift'],
        colorscale='Cividis',  # Use 'Cividis' for a visually appealing and colorblind-friendly colorscale
        colorbar=dict(
            title=dict(
                text='Lift',  # Add "Lift" label to colorbar legend
                font=dict(color='teal')  # Set "Lift" label to teal
            ),
            tickfont=dict(color='plum'),  # Set tick labels to plum
            ticks='outside',
        ),
        showscale=True
    ),
    text=[
        f'Antecedents: {a}<br>'
        f'Consequents: {c}<br>'
        f'Support: {support:.2f}<br>'
        f'Confidence: {confidence:.2f}<br>'
        f'Lift: {lift:.2f}'
        for a, c, support, confidence, lift in zip(rules['antecedents'], rules['consequents'], rules['support'], rules['confidence'], rules['lift'])
    ],
    showlegend=False,
    hoverinfo='text',
    name='Rules'
))

# Update layout with titles and axis labels
fig.update_layout(
    title={
        'text': 'Market Basket Analysis - Support vs. Confidence',
        'font': {'size': 16, 'color': 'teal'}  # Set title font color to teal
    },
    xaxis_title='Support',
    yaxis_title='Confidence',
    showlegend=True,
    template='plotly_white',  # Use a white template for a clean look
    xaxis=dict(
        title_font=dict(size=14, family='Arial', color='teal'),
        tickfont=dict(size=12, color='plum')
    ),
    yaxis=dict(
        title_font=dict(size=14, family='Arial', color='teal'),
        tickfont=dict(size=12, color='plum')
    ),
    paper_bgcolor='#2A2A2A',  # Set background color to a dark gray, close to black
    plot_bgcolor='#2C2C2C'    # Set plot area background color to light blue
)

# Show the interactive plot
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


---

# Interactive Network Graph of Association Rules

In [ ]:
# Create a directed graph
G = nx.DiGraph()

# Add edges with support, confidence, lift, and rule name as edge attributes
for index, row in rules.iterrows():
    antecedent = str(row['antecedents'])  # Ensure it's a string
    consequent = str(row['consequents'])  # Ensure it's a string
    support = row['support']
    confidence = row['confidence']
    lift = row['lift']
    rule_name = f"{antecedent} -> {consequent}"
    G.add_edge(antecedent, consequent, support=support, confidence=confidence, lift=lift, rule_name=rule_name)

    # Add or update node attributes to ensure they have both antecedent and consequent if applicable
    if not G.nodes[antecedent].get('antecedent'):
        G.nodes[antecedent]['antecedent'] = antecedent
    if not G.nodes[antecedent].get('consequent'):
        G.nodes[antecedent]['consequent'] = consequent
    G.nodes[antecedent]['support'] = support
    G.nodes[antecedent]['confidence'] = confidence
    G.nodes[antecedent]['lift'] = lift

    if not G.nodes[consequent].get('antecedent'):
        G.nodes[consequent]['antecedent'] = antecedent
    if not G.nodes[consequent].get('consequent'):
        G.nodes[consequent]['consequent'] = consequent
    G.nodes[consequent]['support'] = support
    G.nodes[consequent]['confidence'] = confidence
    G.nodes[consequent]['lift'] = lift

# Get position of nodes using spring layout
pos = nx.spring_layout(G, k=0.6, iterations=50)

# Create edge traces
edge_trace = []
for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]

    # Create a trace for each edge
    edge_trace.append(
        go.Scatter(
            x=[x0, x1, None],
            y=[y0, y1, None],
            mode='lines',
            line=dict(width=edge[2]['support']*80, color='#FFFFFF'),  # Edge color set to white
            hoverinfo='text',
            text=f"Support: {edge[2]['support']:.2f}<br>Confidence: {edge[2]['confidence']:.2f}<br>Lift: {edge[2]['lift']:.2f}<br>Rule: {edge[2]['rule_name']}",
            opacity=0.6
        )
    )

# Create node trace with chocolate color and detailed hover text
node_trace = go.Scatter(
    x=[pos[node][0] for node in G.nodes()],
    y=[pos[node][1] for node in G.nodes()],
    mode='markers',
    hoverinfo='text',
    text=[f"Antecedent: {G.nodes[node].get('antecedent', 'N/A')}<br>Consequent: {G.nodes[node].get('consequent', 'N/A')}<br>Support: {G.nodes[node].get('support', 'N/A')}<br>Confidence: {G.nodes[node].get('confidence', 'N/A')}<br>Lift: {G.nodes[node].get('lift', 'N/A')}"
          for node in G.nodes()],
    marker=dict(
        showscale=False,
        color='#40E0D0',  # Node color set to turquoise blue
        size=20,
        opacity=0.6,
        line=dict(width=0)  # No border for nodes
    )
)

# Create the figure
fig = go.Figure(data=edge_trace + [node_trace],
                layout=go.Layout(
                    title='Interactive Network Graph of Association Rules',
                    titlefont=dict(size=16, color='#D2B48C'),  # Title color set to leather
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=0, l=0, r=0, t=40),
                    xaxis=dict(
                        showgrid=False,
                        zeroline=False,
                        tickfont=dict(color='#D2B48C'),  # X-axis tick labels color
                        titlefont=dict(color='#D2B48C')  # X-axis title color
                    ),
                    yaxis=dict(
                        showgrid=False,
                        zeroline=False,
                        tickfont=dict(color='#D2B48C'),  # Y-axis tick labels color
                        titlefont=dict(color='#D2B48C')  # Y-axis title color
                    ),
                    paper_bgcolor='#2A2A2A',  # Background color
                    plot_bgcolor='#2A2A2A'    # Plot area color
                ))

# Show the figure
fig.show()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



---

# RFM Analysis

In [ ]:
purchase_df_rfm = purchase_df
purchase_df_rfm['Total_Price'] = purchase_df_rfm['Quantity'] * purchase_df_rfm['Price']

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [ ]:
purchase_df_rfm.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country,Total_Price
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [ ]:
# Define today’s date for recency calculation
today_date = purchase_df_rfm['Date'].max()

# Group by CustomerID to calculate RFM metrics
rfm = purchase_df_rfm.groupby('CustomerID').agg({
    'Date': lambda x: (today_date - x.max()).days,   # Recency
    'BillNo': 'nunique',                             # Frequency (count unique orders)
    'Total_Price': 'sum'                             # Monetary (sum of total price)
})

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [ ]:
# Rename columns for clarity
rfm.columns = ['recency', 'frequency', 'monetary']

# Reset index for further analysis
rfm = rfm.reset_index()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [ ]:
# Check the results
rfm.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



,CustomerID,recency,frequency,monetary
0,12346.0,325,1,77183.60
1,12347.0,1,7,4310.00
2,12349.0,18,1,1757.55
3,12350.0,309,1,334.40
4,12352.0,35,8,2506.04


---

# Cluster Analysis

In [ ]:
# Suppress specific warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

scaler = StandardScaler()
rfm_scaled = scaler.fit_transform(rfm[['recency', 'frequency', 'monetary']])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning:

`should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.



In [ ]:
# Set number of clusters (k) and fit the model
kmeans = KMeans(n_clusters=3, n_init=10, random_state=42)
kmeans.fit(rfm_scaled)

# Add the cluster labels to the original RFM DataFrame
rfm['Cluster'] = kmeans.labels_

# Check the distribution of customers in each cluster
rfm['Cluster'].value_counts()

,count
Cluster,
1,3189
2,1082
0,25


---

# Cluster Profile

In [ ]:
cluster_profile = rfm.groupby('Cluster').agg({
    'recency': 'mean',
    'frequency': 'mean',
    'monetary': 'mean',
    'CustomerID': 'count'  # Count of customers in each cluster
}).rename(columns={'CustomerID': 'num_customers'})

In [ ]:
cluster_profile

,recency,frequency,monetary,num_customers
Cluster,,,,
0,17.880000,58.880000,81979.682000,25
1,40.088115,4.697711,1855.828408,3189
2,245.247689,1.576710,551.568901,1082


---

# Recency vs Frequency by Cluster

In [ ]:
# Define new colors for clusters
colors = {0: '#4169E1',  # Royal Blue
          1: '#DC143C',  # Crimson Red
          2: '#3CB371'}  # Medium Sea Green

# Create a Plotly scatter plot with fixed point size
fig = go.Figure()

for cluster in rfm['Cluster'].unique():
    cluster_data = rfm[rfm['Cluster'] == cluster]
    fig.add_trace(go.Scatter(
        x=cluster_data['recency'],
        y=cluster_data['frequency'],
        mode='markers',
        marker=dict(
            size=12,  # Set fixed size for points
            color=colors[cluster],
            line=dict(width=1, color='white')  # Optional: add a white border to points
        ),
        name=f'Cluster {cluster}'
    ))

# Update layout to match desired style
fig.update_layout(
    paper_bgcolor='#2A2A2A',  # Background color
    plot_bgcolor='#2A2A2A',   # Plot area color
    title='Recency vs Frequency by Cluster',
    title_font=dict(size=16, color='Teal'),  # Title color
    xaxis_title='Recency',
    xaxis_title_font=dict(size=14, color='Teal'),  # X-axis label color
    yaxis_title='Frequency',
    yaxis_title_font=dict(size=14, color='Teal'),  # Y-axis label color
    legend_title_font=dict(size=13, color='White'),  # Legend title color
    legend_font=dict(size=11, color='Plum'),  # Legend text color
    xaxis=dict(
        showgrid=True,
        zeroline=False,
        showline=False,
        linecolor='#FFFFFF',  # Outline color
        tickfont=dict(color='Plum'),  # Tick label color

    ),
    yaxis=dict(
        showgrid=True,
        zeroline=False,
        showline=False,
        linecolor='#FFFFFF',  # Outline color
        tickfont=dict(color='Plum')  # Tick label color
    ),
    margin=dict(l=40, r=40, t=40, b=40)  # Adjust margins
)

# Show the interactive plot
fig.show()

___

# **Market Basket Analysis for United Kingdom**

# Data Exploration and Preparation

In [ ]:
purchase_df.head()

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country,Total_Price
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [ ]:
purchase_df['Country'].unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'Switzerland', 'Spain', 'Poland', 'Portugal', 'Italy',
       'Belgium', 'Lithuania', 'Japan', 'Iceland', 'Sweden', 'Austria',
       'Greece', 'Singapore', 'Lebanon', 'United Arab Emirates', 'Israel',
       'Saudi Arabia', 'Unspecified', 'Brazil', 'USA', 'Bahrain', 'Malta',
       'RSA'], dtype=object)

In [ ]:
purchase_df['Country'].value_counts()

,count
Country,
United Kingdom,354321
Germany,9040
France,8341
Spain,2484
Netherlands,2359
Belgium,2031
Switzerland,1841
Portugal,1462
Australia,1182


In [ ]:
# Filter by a specific country if required
purchase_df_UK = purchase_df[purchase_df['Country'] == 'United Kingdom']  # optional filtering by country

In [ ]:
purchase_df_UK.info()

<class 'pandas.core.frame.DataFrame'>
Index: 354321 entries, 0 to 522048
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   BillNo       354321 non-null  object        
 1   Itemname     354321 non-null  object        
 2   Quantity     354321 non-null  int64         
 3   Date         354321 non-null  datetime64[ns]
 4   Price        354321 non-null  float64       
 5   CustomerID   354321 non-null  object        
 6   Country      354321 non-null  object        
 7   Total_Price  354321 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 24.3+ MB


In [ ]:
transaction_UK = purchase_df_UK.groupby(['BillNo', 'Date'])['Itemname'].apply(lambda x: ', '.join(x)).reset_index()
transaction_UK.head()

,BillNo,Date,Itemname
0,536365,2010-12-01 08:26:00,"WHITE HANGING HEART T-LIGHT HOLDER, WHITE META..."
1,536366,2010-12-01 08:28:00,"HAND WARMER UNION JACK, HAND WARMER RED POLKA DOT"
2,536367,2010-12-01 08:34:00,"ASSORTED COLOUR BIRD ORNAMENT, POPPY'S PLAYHOU..."
3,536368,2010-12-01 08:34:00,"JAM MAKING SET WITH JARS, RED COAT RACK PARIS ..."
4,536369,2010-12-01 08:35:00,BATH BUILDING BLOCK WORD


In [ ]:
# Drop unnecessary columns
transaction_UK.drop(columns=['BillNo', 'Date'], inplace=True)

In [ ]:
# Split the 'Itemname' column into separate rows
transaction_UK = transaction_UK.assign(Itemname=transaction_UK['Itemname'].str.split(', ')).explode('Itemname')

# Create a crosstab (binary matrix) of items per transaction
basket_encoded_UK = pd.crosstab(index=transaction_UK.index, columns=transaction_UK['Itemname'])

# Group by the transaction index and convert the counts of items done by pd.crosstab() into a simple presence or absence
basket_encoded_UK = basket_encoded_UK.groupby(level=0).apply(lambda x: x > 0)

---

# Association Rule Mining


In [ ]:
# Association Rule Mining
frequent_itemsets_UK = apriori(basket_encoded_UK, min_support=0.01, use_colnames=True)
rules_UK = association_rules(frequent_itemsets_UK, metric="confidence", min_threshold=0.6, num_itemsets= frequent_itemsets_UK)

In [ ]:
# Display information of the rules
print("Association Rules:")
rules_UK.head()

Association Rules:


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(ALARM CLOCK BAKELIKE CHOCOLATE),(ALARM CLOCK BAKELIKE GREEN),0.016494,0.041384,0.010916,0.661818,15.992021,1.0,0.010233,2.834616,0.953190,0.232439,0.647219,0.462793
1,(ALARM CLOCK BAKELIKE CHOCOLATE),(ALARM CLOCK BAKELIKE RED),0.016494,0.045463,0.011276,0.683636,15.037294,1.0,0.010526,3.017216,0.949154,0.222485,0.668569,0.465829
2,(ALARM CLOCK BAKELIKE ORANGE),(ALARM CLOCK BAKELIKE GREEN),0.018773,0.041384,0.011396,0.607029,14.668102,1.0,0.010619,2.439404,0.949653,0.233702,0.590064,0.441196
3,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.041384,0.045463,0.027230,0.657971,14.472758,1.0,0.025348,2.790808,0.971093,0.456740,0.641681,0.628458
4,(ALARM CLOCK BAKELIKE IVORY),(ALARM CLOCK BAKELIKE RED),0.025430,0.045463,0.016914,0.665094,14.629443,1.0,0.015757,2.850168,0.955955,0.313333,0.649143,0.518563


---

# Market Basket Analysis - Support vs. Confidence

In [ ]:
# Convert frozensets to lists for serialization
rules_UK['antecedents'] = rules_UK['antecedents'].apply(list)
rules_UK['consequents'] = rules_UK['consequents'].apply(list)

# Create a scatter plot using plotly.graph_objects
fig = go.Figure()

# Add scatter trace
fig.add_trace(go.Scatter(
    x=rules_UK['support'],
    y=rules_UK['confidence'],
    mode='markers',
    marker=dict(
        size=rules_UK['lift'] * 1,  # Scale size by lift for better visibility
        color=rules_UK['lift'],
        colorscale='Cividis',  # Use 'Cividis' for a visually appealing and colorblind-friendly colorscale
        colorbar=dict(
            title=dict(
                text='Lift',  # Add "Lift" label to colorbar legend
                font=dict(color='teal')  # Set "Lift" label to teal
            ),
            tickfont=dict(color='plum'),  # Set tick labels to plum
            ticks='outside',
        ),
        showscale=True
    ),
    text=[
        f'Antecedents: {a}<br>'
        f'Consequents: {c}<br>'
        f'Support: {support:.2f}<br>'
        f'Confidence: {confidence:.2f}<br>'
        f'Lift: {lift:.2f}'
        for a, c, support, confidence, lift in zip(rules_UK['antecedents'], rules_UK['consequents'], rules_UK['support'], rules_UK['confidence'], rules_UK['lift'])
    ],
    showlegend=False,
    hoverinfo='text',
    name='Rules'
))

# Update layout with titles and axis labels
fig.update_layout(
    title={
        'text': 'Market Basket Analysis - Support vs. Confidence',
        'font': {'size': 16, 'color': 'teal'}  # Set title font color to teal
    },
    xaxis_title='Support',
    yaxis_title='Confidence',
    showlegend=True,
    template='plotly_white',  # Use a white template for a clean look
    xaxis=dict(
        title_font=dict(size=14, family='Arial', color='teal'),
        tickfont=dict(size=12, color='plum')
    ),
    yaxis=dict(
        title_font=dict(size=14, family='Arial', color='teal'),
        tickfont=dict(size=12, color='plum')
    ),
    paper_bgcolor='#2A2A2A',  # Set background color to a dark gray, close to black
    plot_bgcolor='#2C2C2C'    # Set plot area background color to light blue
)

# Show the interactive plot
fig.show()

---

# Interactive Network Graph of Association Rules

In [ ]:
# Create a directed graph
G = nx.DiGraph()

# Add edges with support, confidence, lift, and rule name as edge attributes
for index, row in rules_UK.iterrows():
    antecedent = str(row['antecedents'])  # Ensure it's a string
    consequent = str(row['consequents'])  # Ensure it's a string
    support = row['support']
    confidence = row['confidence']
    lift = row['lift']
    rule_name = f"{antecedent} -> {consequent}"
    G.add_edge(antecedent, consequent, support=support, confidence=confidence, lift=lift, rule_name=rule_name)

    # Add or update node attributes to ensure they have both antecedent and consequent if applicable
    if not G.nodes[antecedent].get('antecedent'):
        G.nodes[antecedent]['antecedent'] = antecedent
    if not G.nodes[antecedent].get('consequent'):
        G.nodes[antecedent]['consequent'] = consequent
    G.nodes[antecedent]['support'] = support
    G.nodes[antecedent]['confidence'] = confidence
    G.nodes[antecedent]['lift'] = lift

    if not G.nodes[consequent].get('antecedent'):
        G.nodes[consequent]['antecedent'] = antecedent
    if not G.nodes[consequent].get('consequent'):
        G.nodes[consequent]['consequent'] = consequent
    G.nodes[consequent]['support'] = support
    G.nodes[consequent]['confidence'] = confidence
    G.nodes[consequent]['lift'] = lift

# Get position of nodes using spring layout
pos = nx.spring_layout(G, k=0.4, iterations=50)

# Create edge traces
edge_trace = []
for edge in G.edges(data=True):
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]

    # Create a trace for each edge
    edge_trace.append(
        go.Scatter(
            x=[x0, x1, None],
            y=[y0, y1, None],
            mode='lines',
            line=dict(width=edge[2]['support']*80, color='#FFFFFF'),  # Edge color set to white
            hoverinfo='text',
            text=f"Support: {edge[2]['support']:.2f}<br>Confidence: {edge[2]['confidence']:.2f}<br>Lift: {edge[2]['lift']:.2f}<br>Rule: {edge[2]['rule_name']}",
            opacity=0.6
        )
    )

# Create node trace with chocolate color and detailed hover text
node_trace = go.Scatter(
    x=[pos[node][0] for node in G.nodes()],
    y=[pos[node][1] for node in G.nodes()],
    mode='markers',
    hoverinfo='text',
    text=[f"Antecedent: {G.nodes[node].get('antecedent', 'N/A')}<br>Consequent: {G.nodes[node].get('consequent', 'N/A')}<br>Support: {G.nodes[node].get('support', 'N/A')}<br>Confidence: {G.nodes[node].get('confidence', 'N/A')}<br>Lift: {G.nodes[node].get('lift', 'N/A')}"
          for node in G.nodes()],
    marker=dict(
        showscale=False,
        color='#40E0D0',  # Node color set to turquoise blue
        size=20,
        opacity=0.6,
        line=dict(width=0)  # No border for nodes
    )
)

# Create the figure
fig = go.Figure(data=edge_trace + [node_trace],
                layout=go.Layout(
                    title='Interactive Network Graph of Association Rules',
                    titlefont=dict(size=16, color='#D2B48C'),  # Title color set to leather
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=0, l=0, r=0, t=40),
                    xaxis=dict(
                        showgrid=False,
                        zeroline=False,
                        tickfont=dict(color='#D2B48C'),  # X-axis tick labels color
                        titlefont=dict(color='#D2B48C')  # X-axis title color
                    ),
                    yaxis=dict(
                        showgrid=False,
                        zeroline=False,
                        tickfont=dict(color='#D2B48C'),  # Y-axis tick labels color
                        titlefont=dict(color='#D2B48C')  # Y-axis title color
                    ),
                    paper_bgcolor='#2A2A2A',  # Background color
                    plot_bgcolor='#2A2A2A'    # Plot area color
                ))

# Show the figure
fig.show()

---

# RFM Analysis

In [ ]:
# Filter by a specific country if required
purchase_df_UK = purchase_df[purchase_df['Country'] == 'United Kingdom']  # optional filtering by country

In [ ]:
purchase_df_UK_rfm = purchase_df_UK.copy()  # Ensure we're working with a copy
purchase_df_UK_rfm.loc[:, 'Total_Price'] = purchase_df_UK_rfm['Quantity'] * purchase_df_UK_rfm['Price']

In [ ]:
purchase_df_UK_rfm.head()

,BillNo,Itemname,Quantity,Date,Price,CustomerID,Country,Total_Price
0,536365,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [ ]:
# Define today’s date for recency calculation
today_date = purchase_df_UK_rfm['Date'].max()

# Group by CustomerID to calculate RFM metrics
rfm_UK = purchase_df_UK_rfm.groupby('CustomerID').agg({
    'Date': lambda x: (today_date - x.max()).days,   # Recency
    'BillNo': 'nunique',                             # Frequency (count unique orders)
    'Total_Price': 'sum'                             # Monetary (sum of total price)
})

In [ ]:
# Rename columns for clarity
rfm_UK.columns = ['recency', 'frequency', 'monetary']

# Reset index for further analysis
rfm_UK = rfm_UK.reset_index()

In [ ]:
# Check the results
rfm_UK.head()

,CustomerID,recency,frequency,monetary
0,12346.0,325,1,77183.60
1,12747.0,1,11,4196.01
2,12748.0,0,209,33719.73
3,12749.0,3,5,4090.88
4,12820.0,2,4,942.34


---

# Cluster Analysis

In [ ]:
# Suppress specific warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

scaler = StandardScaler()
rfm_UK_scaled = scaler.fit_transform(rfm_UK[['recency', 'frequency', 'monetary']])

In [ ]:
# Set number of clusters (k) and fit the model
kmeans = KMeans(n_clusters=3, n_init=10, random_state=42)
kmeans.fit(rfm_UK_scaled)

# Add the cluster labels to the original RFM DataFrame
rfm_UK['Cluster'] = kmeans.labels_

# Check the distribution of customers in each cluster
rfm_UK['Cluster'].value_counts()

,count
Cluster,
0,2901
1,996
2,23


In [ ]:
cluster_profile_UK = rfm_UK.groupby('Cluster').agg({
    'recency': 'mean',
    'frequency': 'mean',
    'monetary': 'mean',
    'CustomerID': 'count'  # Count of customers in each cluster
}).rename(columns={'CustomerID': 'num_customers'})

In [ ]:
cluster_profile_UK

,recency,frequency,monetary,num_customers
Cluster,,,,
0,39.747673,4.716649,1785.031145,2901
1,242.777108,1.591365,487.595081,996
2,18.391304,59.913043,71494.413043,23


---

# Recency vs Frequency by Cluster

In [ ]:
# Define new colors for clusters
colors = {0: '#4169E1',  # Royal Blue
          1: '#DC143C',  # Crimson Red
          2: '#3CB371'}  # Medium Sea Green

# Create a Plotly scatter plot with fixed point size
fig = go.Figure()

for cluster in rfm_UK['Cluster'].unique():
    cluster_data = rfm_UK[rfm_UK['Cluster'] == cluster]
    fig.add_trace(go.Scatter(
        x=cluster_data['recency'],
        y=cluster_data['frequency'],
        mode='markers',
        marker=dict(
            size=12,  # Set fixed size for points
            color=colors[cluster],
            line=dict(width=1, color='white')  # Optional: add a white border to points
        ),
        name=f'Cluster {cluster}'
    ))

# Update layout to match desired style
fig.update_layout(
    paper_bgcolor='#2A2A2A',  # Background color
    plot_bgcolor='#2A2A2A',   # Plot area color
    title='Recency vs Frequency by Cluster',
    title_font=dict(size=16, color='Teal'),  # Title color
    xaxis_title='Recency',
    xaxis_title_font=dict(size=14, color='Teal'),  # X-axis label color
    yaxis_title='Frequency',
    yaxis_title_font=dict(size=14, color='Teal'),  # Y-axis label color
    legend_title_font=dict(size=13, color='White'),  # Legend title color
    legend_font=dict(size=11, color='Plum'),  # Legend text color
    xaxis=dict(
        showgrid=True,
        zeroline=False,
        showline=False,
        linecolor='#FFFFFF',  # Outline color
        tickfont=dict(color='Plum')  # Tick label color
    ),
    yaxis=dict(
        showgrid=True,
        zeroline=False,
        showline=False,
        linecolor='#FFFFFF',  # Outline color
        tickfont=dict(color='Plum')  # Tick label color
    ),
    margin=dict(l=40, r=40, t=40, b=40)  # Adjust margins
)

# Show the interactive plot
fig.show()